In [ ]:
import requests
import urllib
from datetime import datetime
from fake_useragent import UserAgent
import time


keyword = 'iPhone'
headers = {
    'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.150 Safari/537.36 Edg/88.0.705.68',
    'x-api-source': 'pc',
    'referer': f'https://shopee.tw/search?keyword={urllib.parse.quote(keyword)}'
}

s = requests.Session()
url = 'https://shopee.tw/api/v4/search/product_labels'
r = s.get(url, headers=headers)

base_url = 'https://shopee.tw/api/v2/search_items/'
query = f"by=relevancy&conditions=used&keyword={keyword}&limit=100&newest=300&order=desc&page_type=search&price_min=2000&scenario=PAGE_GLOBAL_SEARCH&skip_autocorrect=1&version=2"


"%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20iphone" 
"%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20pixel" 
"%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20samsung"
"%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20oppo"
"%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20%E5%B0%8F%E7%B1%B3" # 小米
"%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20asus"
"%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20realme"

url = base_url + '?' + query
r = s.get(url, headers=headers)
if r.status_code == requests.codes.ok:
    data = r.json()

In [ ]:
from pprint import pprint
print(len(data["items"]))

In [ ]:
pprint(data["items"][0])

In [ ]:
import requests
import urllib
from datetime import datetime
from fake_useragent import UserAgent
import time
import csv 
import random

proxy_lst = []
with open("proxies.csv") as file:
    for row in file:
        proxy_lst.append(row.strip())
proxy_lst  

In [ ]:
proxies = []
for p in proxy_lst:
    p_lst = p.split(":")
    proxies.append("http://"+p_lst[2]+":"+p_lst[3]+"@"+p_lst[0]+":"+p_lst[1])  

In [ ]:
with open('proxies.csv', 'w', newline='') as csvfile:
  # 建立 CSV 檔寫入器
  writer = csv.writer(csvfile)

  # 寫入一列資料
  for p in proxies:
      writer.writerow([p])

In [ ]:
def gen_header_search(keyword):
    ua = UserAgent()
    fakeua = ua.random
    headers = {
    'user-agent': fakeua,
    'x-api-source': 'pc',
    'referer': f'https://shopee.tw/search?keyword={urllib.parse.quote(keyword)}'
    }
    return headers

def gen_header(keyword, referer):
    ua = UserAgent()
    fakeua = ua.random
    headers = {
    'user-agent': fakeua,
    'x-api-source': 'pc',
    'referer': referer
    }
    return headers

In [ ]:
data_lst = []
end = 200
step = 50
# keywords = ["%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20iphone", 
# "%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20pixel",
# "%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20samsung",
# "%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20oppo",
# "%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20%E5%B0%8F%E7%B1%B3", # 小米
# "%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20asus",
# "%E4%BA%8C%E6%89%8B%E6%89%8B%E6%A9%9F%20realme"]
keyword = "二手手機"
init = time.time()
random.shuffle(proxy_lst)
proxy_chosen = proxy_lst[:int(end/step)]
i = 0
while i < end:
    p = proxy_chosen[int(i/step)%100]
    proxies = {"http": p, "https": p}
    headers = gen_header_search(keyword)
    s = requests.Session()
    url = 'https://shopee.tw/api/v4/search/product_labels'
    r = s.get(url, headers=headers, proxies=proxies)

    base_url = 'https://shopee.tw/api/v2/search_items/'
    query = f"by=relevancy&conditions=used&keyword={keyword}&limit={step}&newest={i}&order=desc&page_type=search&min_price=2000&scenario=PAGE_GLOBAL_SEARCH&skip_autocorrect=1&version=2"

    url = base_url + '?' + query
    r = s.get(url, headers=headers, proxies=proxies)
    print(time.time()-init)
    if r.status_code == requests.codes.ok:
        data = r.json()
        items = data["items"]
        if items[-1]["ctime"] > datetime.timestamp(datetime.now()):
            print("Take a rest!")
            time.sleep(30)
        else:   
            for d in items:
                if int(d["price"]/100000) > 2000:
                    start = time.time()
                    rf = f'https://shopee.tw/product/{d["shopid"]}/{d["itemid"]}'
                    headers = gen_header(keyword, rf)
                    url = f'https://shopee.tw/api/v2/item/get?itemid={d["itemid"]}&shopid={d["shopid"]}'
                    r = requests.get(url, headers=headers)
                    item = r.json()["item"]
                    data_lst.append(
                        (int(d["price"]/100000), # price
                         d["name"], # title
                         item["description"], # box
                         (item["status"] == "sold_out"), # sold
                         datetime.utcfromtimestamp(d["ctime"]).strftime('%Y-%m-%d %H:%M:%S'), # created_at
                         f'https://shopee.tw/product/{d["shopid"]}/{d["itemid"]}')) # link)
                    print(time.time()-start)
            i += step
    else:
        i += step
    print(i, len(data_lst))
print(time.time()-init)

In [ ]:
# items = sorted(data_lst, key=lambda x: x[4])
for l in sorted(data_lst, key=lambda x:x[4]):
    print(l[3], l[4], l[5])

In [ ]:
data_lst[0]

In [ ]:
for d in data_lst:
    print(f'https://shopee.tw/product/{d["shopid"]}/{d["itemid"]}')

In [ ]:
f'https://shopee.tw/product/{items[-1]["shopid"]}/{items[-1]["itemid"]}'

In [ ]:
items[-1]

In [ ]:
items[-1]["images"]

In [ ]:
items[-1]["image"]

In [ ]:
items[-1]["ctime"]

In [ ]:
46c7a365d3f00de720b33777876 2b3e4
8df065f71d6a7b37eca1791b793 f8ff4
f925f525da18885fdcce030b34e 50814
d6699bdcd7d8295254356346df8 5e12a

In [ ]:
from pprint import pprint
import json 
data = json.loads('{"item":{"itemid":5285372925,"price_max_before_discount":-1,"item_status":"normal","can_use_wholesale":false,"brand_id":null,"show_free_shipping":false,"estimated_days":3,"is_hot_sales":null,"is_slash_price_item":false,"upcoming_flash_sale":null,"slash_lowest_price":null,"is_partial_fulfilled":false,"condition":4,"show_original_guarantee":false,"add_on_deal_info":null,"is_non_cc_installment_payment_eligible":false,"categories":[{"display_name":"3C","catid":69,"image":null,"no_sub":false,"is_default_subcat":false,"block_buyer_platform":null},{"display_name":"","catid":298,"image":null,"no_sub":true,"is_default_subcat":true,"block_buyer_platform":null},{"display_name":"","catid":5787,"image":null,"no_sub":true,"is_default_subcat":true,"block_buyer_platform":null}],"ctime":1618148679,"name":"iphoneXS 256g","show_shopee_verified_label":false,"userid":null,"size_chart":null,"is_pre_order":false,"service_by_shopee_flag":null,"historical_sold":0,"reference_item_id":"","recommendation_info":null,"bundle_deal_info":null,"bundle_deal_id":0,"has_lowest_price_guarantee":false,"shipping_icon_type":0,"overall_purchase_limit":{"order_max_purchase_limit":0,"item_overall_quota":null,"overall_purchase_limit":null,"end_date":null,"start_date":null},"images":["d6699bdcd7d8295254356346df85e12a","f925f525da18885fdcce030b34e50814","46c7a365d3f00de720b337778762b3e4","8df065f71d6a7b37eca1791b793f8ff4"],"price_before_discount":0,"cod_flag":0,"catid":69,"is_official_shop":false,"coin_earn_label":null,"hashtag_list":null,"sold":0,"makeup":null,"item_rating":{"rating_star":0.0,"rating_count":[0,0,0,0,0,0],"rcount_with_image":0,"rcount_with_context":0},"show_official_shop_label_in_title":false,"discount":null,"reason":null,"label_ids":[],"has_group_buy_stock":false,"other_stock":1,"deep_discount":null,"attributes":[{"is_pending_qc":false,"idx":0,"value":"Apple ","id":9547,"is_timestamp":false,"name":""}],"last_active_time":null,"badge_icon_type":0,"liked":false,"is_on_flash_sale":null,"cmt_count":0,"is_live_streaming_price":null,"image":"d6699bdcd7d8295254356346df85e12a","recommendation_algorithm":null,"is_cc_installment_payment_eligible":false,"shopid":1444784,"normal_stock":1,"video_info_list":[],"installment_plans":null,"view_count":1043,"voucher_info":null,"current_promotion_has_reserve_stock":false,"liked_count":12,"show_official_shop_label":false,"price_min_before_discount":-1,"show_discount":0,"preview_info":null,"flag":65536,"exclusive_price_info":null,"current_promotion_reserved_stock":0,"wholesale_tier_list":[],"group_buy_info":null,"shopee_verified":false,"item_has_post":false,"hidden_price_display":null,"transparent_background_image":"","welcome_package_info":null,"discount_stock":0,"coin_info":{"spend_cash_unit":100000,"coin_earn_items":[]},"is_adult":false,"currency":"TWD","raw_discount":0,"is_preferred_plus_seller":false,"is_category_failed":false,"price_min":600000000,"can_use_bundle_deal":false,"cb_option":0,"brand":null,"stock":1,"status":1,"price_max":600000000,"spl_info":null,"is_group_buy_item":null,"description":"iphoneXS 256g","flash_sale":null,"models":[{"itemid":5285372925,"status":1,"current_promotion_reserved_stock":0,"name":"","promotionid":0,"price":600000000,"price_stocks":[{"model_id":74783133950,"stockout_time":0,"region":"TW","rebate":null,"price":600000000,"promotion_type":0,"allocated_stock":null,"shop_id":1444784,"end_time":null,"stock_breakdown_by_location":[],"item_id":5285372925,"promotion_id":0,"purchase_limit":null,"start_time":null,"stock":1}],"current_promotion_has_reserve_stock":false,"currency":"TWD","normal_stock":1,"extinfo":{"seller_promotion_limit":null,"has_shopee_promo":null,"group_buy_info":null,"holiday_mode_old_stock":null,"tier_index":[0],"seller_promotion_refresh_time":0},"has_gimmick_tag":false,"price_before_discount":0,"modelid":74783133950,"sold":0,"stock":1}],"has_low_fulfillment_rate":false,"price":600000000,"shop_location":"","tier_variations":[{"images":[],"properties":[],"type":0,"name":"","options":[""]}],"min_purchase_limit":1,"can_use_cod":null,"makeups":null,"welcome_package_type":0,"show_official_shop_label_in_normal_position":null,"is_alcohol_product":false,"item_type":0,"spl_installment_tenure":null,"show_recycling_info":false},"version":"98e3bbe5b302c1f9768100015ca1725f","data":null,"error_msg":null,"error":null}')

In [ ]:
pprint(data)

In [ ]:
print(datetime.utcfromtimestamp(1618148679).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
url = "https://shopee.tw/api/v2/item/get?itemid=5285372925&shopid=1444784"
r = s.get(url, headers=headers, proxies=proxies)
data = r.json()

In [ ]:
pprint(data)

In [ ]:
print(datetime.utcfromtimestamp(1612367777).strftime('%Y-%m-%d %H:%M:%S'))

In [ ]:
cookie: _gcl_au=1.1.1353058702.1617528965; _fbp=fb.1.1617528965003.1799836385; csrftoken=pgtYtz8G2vj5clXuB12HqD3NPpN1EaSu; SPC_IA=-1; SPC_EC=-; SPC_U=-; SPC_F=eVcYNw5qsxs00YYhGHv5YKlkwtZYkDLh; REC_T_ID=416f8f82-a523-11eb-a4d9-f8f21e1a2550; __BWfp=c1619285641166x18d9dff8a; _med=refer; welcomePkgShown=true; SC_DFP=MR8fIQbimAUVnqBtqjULPQaoQNVuZfl1; SPC_IVS=; SPC_SC_UD=; UYOMAPJWEMDGJ=; SPC_SC_TK=; SPC_SI=mall.4UPJ6X7RwY6QeQPTQSHIcumXQw1XXLVi; _gid=GA1.2.393335809.1622618782; _ga=GA1.1.2024272600.1619285641; SPC_R_T_ID="OMQHPHPBW5XPfqwvezrJJTDJaQ7VQm0HQ6xvD1IfXSLU/8RL1bPHyMdOl3D6y9TO+iiwa1d6QAFoaN0zpLzUpxdifN0iIW1ssZLiRj7X1Yo="; SPC_T_IV="h6tWhkx1ZemShVjtsSv2Sg=="; SPC_R_T_IV="h6tWhkx1ZemShVjtsSv2Sg=="; SPC_T_ID="OMQHPHPBW5XPfqwvezrJJTDJaQ7VQm0HQ6xvD1IfXSLU/8RL1bPHyMdOl3D6y9TO+iiwa1d6QAFoaN0zpLzUpxdifN0iIW1ssZLiRj7X1Yo="; _gali=main; _ga_RPSBE3TQZZ=GS1.1.1622623629.11.1.1622628990.60

In [ ]:
cookie: _gcl_au=1.1.1353058702.1617528965; _fbp=fb.1.1617528965003.1799836385; csrftoken=pgtYtz8G2vj5clXuB12HqD3NPpN1EaSu; SPC_IA=-1; SPC_EC=-; SPC_U=-; SPC_F=eVcYNw5qsxs00YYhGHv5YKlkwtZYkDLh; REC_T_ID=416f8f82-a523-11eb-a4d9-f8f21e1a2550; __BWfp=c1619285641166x18d9dff8a; _med=refer; welcomePkgShown=true; SC_DFP=MR8fIQbimAUVnqBtqjULPQaoQNVuZfl1; SPC_IVS=; SPC_SC_UD=; UYOMAPJWEMDGJ=; SPC_SC_TK=; SPC_SI=mall.4UPJ6X7RwY6QeQPTQSHIcumXQw1XXLVi; _gid=GA1.2.393335809.1622618782; AMP_TOKEN=%24NOT_FOUND; _ga=GA1.1.2024272600.1619285641; SPC_R_T_ID="RVNbL2WgzTjEdEVv6U/LiRRgQDko8ndOFmZVGWmDdYS/BD1EMLRbptCpHJ+d8pHRsOCLbgSjMTcIHxE10d+egfvOFP7rA8Yyfa7hVxdyeL4="; SPC_T_IV="gTnLB5ikTgbTuumXukMQ8g=="; SPC_R_T_IV="gTnLB5ikTgbTuumXukMQ8g=="; SPC_T_ID="RVNbL2WgzTjEdEVv6U/LiRRgQDko8ndOFmZVGWmDdYS/BD1EMLRbptCpHJ+d8pHRsOCLbgSjMTcIHxE10d+egfvOFP7rA8Yyfa7hVxdyeL4="; _ga_RPSBE3TQZZ=GS1.1.1622643240.13.1.1622644278.54